Función 4:
def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [66]:
import pandas as pd

In [78]:
df1 = pd.read_csv('user_review_sentiment.csv')

In [79]:
# Reemplaza 'df' con el nombre real de tu DataFrame y 'columna_texto' con el nombre de la columna que deseas modificar
df1['posted'] = df1['posted'].str.replace('Posted', '', case=False)


# Reemplaza 'df' con el nombre real de tu DataFrame y 'columna_texto' con el nombre de la columna que deseas modificar
palabras_a_eliminar = [    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December"]

for palabra in palabras_a_eliminar:
    df1['posted'] = df1['posted'].str.replace(palabra, ' ', case=False)

df1['posted'] = df1['posted'].str.replace(',', '-', case=False)
df1['posted'] = df1['posted'].str.replace('.', '', case=False)

# Divide la columna 'posted' por el guion y selecciona la segunda parte
df1['posted'] = df1['posted'].str.split('-').str[1]

# Convierte la columna 'posted' a tipo numérico si es necesario
df1['posted'] = pd.to_numeric(df1['posted'], errors='coerce')


In [80]:
df1.dropna(subset = 'posted', inplace= True)

In [81]:
df1['posted'] = df1['posted'].astype(int)

In [82]:
df1_recommend_false = df1[df1['recommend'] == False]

In [83]:
df1_recommend_false = df1_recommend_false [[ 'posted', 'item_id', 'sentiment']]

In [84]:
df1_recommend_false.rename(columns={'item_id': 'id'}, inplace=True)

In [85]:
df1_recommend_false

,posted,id,sentiment
48,2014,33440.0,0
62,2015,359320.0,0
68,2014,570.0,1
84,2015,383080.0,1
122,2015,417860.0,1
...,...,...,...
59087,2015,295110.0,0
59144,2015,311210.0,0
59166,2015,305920.0,0
59225,2015,370240.0,0


In [86]:
df1_recommend_false['sentiment'] = df1_recommend_false['sentiment'].replace(0, 2)


In [87]:
df1_recommend_false

,posted,id,sentiment
48,2014,33440.0,2
62,2015,359320.0,2
68,2014,570.0,1
84,2015,383080.0,1
122,2015,417860.0,1
...,...,...,...
59087,2015,295110.0,2
59144,2015,311210.0,2
59166,2015,305920.0,2
59225,2015,370240.0,2


In [88]:
df_menos_recomendado_x_anio = df1_recommend_false.groupby(['posted', 'id'])['sentiment'].sum().reset_index()

# Ordena de mayor a menor
df_menos_recomendado_x_anio = df_menos_recomendado_x_anio.sort_values(by='sentiment', ascending=False)

In [89]:
df_menos_recomendado_x_anio

,posted,id,sentiment
1044,2015,218620.0,265
827,2015,730.0,228
1056,2015,221100.0,133
480,2014,221100.0,112
187,2014,730.0,91
...,...,...,...
193,2014,2450.0,1
1495,2015,344740.0,1
191,2014,1900.0,1
815,2015,30.0,1


In [90]:
df2 = pd.read_csv('steam_games.csv')
df2 = df2[['app_name','id']]
df2

C:\Users\Heidi\AppData\Local\Temp\ipykernel_19164\2830986885.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('steam_games.csv')


,app_name,id
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
120440,Colony On Mars,773640.0
120441,LOGistICAL: South Africa,733530.0
120442,Russian Roads,610660.0
120443,EXIT 2 - Directions,658870.0


In [91]:
df_menos_recomendado_x_anio = df_menos_recomendado_x_anio.merge(df2, on='id', how='left')
df_menos_recomendado_x_anio

,posted,id,sentiment,app_name
0,2015,218620.0,265,PAYDAY 2
1,2015,730.0,228,Counter-Strike: Global Offensive
2,2015,221100.0,133,DayZ
3,2014,221100.0,112,DayZ
4,2014,730.0,91,Counter-Strike: Global Offensive
...,...,...,...,...
1640,2014,2450.0,1,Bloody Good Time
1641,2015,344740.0,1,CRYPTARK
1642,2014,1900.0,1,Earth 2160
1643,2015,30.0,1,Day of Defeat


In [92]:
df_menos_recomendado_x_anio.dropna(subset=["app_name"], inplace=True)


In [93]:
años = df_menos_recomendado_x_anio['posted'].unique()
años


array([2015, 2014, 2013, 2011, 2010, 2012])

In [94]:
años_interés = [2014, 2015, 2013, 2012, 2011, 2010]

# Crear un DataFrame vacío para almacenar los resultados
result_df = pd.DataFrame(columns=['posted', 'id', 'sentiment'])

# Iterar a través de cada año y obtener los tres IDs con el puntaje de "sentiment" NEGATIVO más alto
for año in años_interés:
    df_año = df_menos_recomendado_x_anio[df_menos_recomendado_x_anio['posted'] == año]
    top_3_ids = df_año.nlargest(3, 'sentiment')
    result_df = pd.concat([result_df, top_3_ids], ignore_index=True)


# Restablecer el índice del DataFrame resultante
result_df.reset_index(drop=True, inplace=True)

# Muestra el resultado
print(result_df)


   posted        id sentiment                          app_name
0    2014  221100.0       112                              DayZ
1    2014     730.0        91  Counter-Strike: Global Offensive
2    2014     570.0        70                            Dota 2
3    2015  218620.0       265                          PAYDAY 2
4    2015     730.0       228  Counter-Strike: Global Offensive
5    2015  221100.0       133                              DayZ
6    2013  209160.0        26             Call of Duty®: Ghosts
7    2013     440.0        21                   Team Fortress 2
8    2013  224540.0        19     Ace of Spades: Battle Builder
9    2012  209870.0         2           Blacklight: Retribution
10   2012     440.0         2                   Team Fortress 2
11   2012     570.0         2                            Dota 2
12   2011   18700.0         4                  And Yet It Moves
13   2011     440.0         3                   Team Fortress 2
14   2011   33460.0         2           

In [95]:
result_df.to_csv('Funcion_4_jup.csv')

In [96]:
users_not_recommend = pd.read_csv('Funcion_4_jup.csv')
users_not_recommend

,Unnamed: 0,posted,id,sentiment,app_name
0,0,2014,221100.0,112,DayZ
1,1,2014,730.0,91,Counter-Strike: Global Offensive
2,2,2014,570.0,70,Dota 2
3,3,2015,218620.0,265,PAYDAY 2
4,4,2015,730.0,228,Counter-Strike: Global Offensive
5,5,2015,221100.0,133,DayZ
6,6,2013,209160.0,26,Call of Duty®: Ghosts
7,7,2013,440.0,21,Team Fortress 2
8,8,2013,224540.0,19,Ace of Spades: Battle Builder
9,9,2012,209870.0,2,Blacklight: Retribution


In [97]:
def UsersNotRecommend(posted):
    # Filtrar el DataFrame por el año dado
    df_año = users_not_recommend[users_not_recommend['posted'] == posted]

    # Aplicar las condiciones para obtener el top 3
    top_3_df = df_año[(df_año['sentiment'] >= 0) & (df_año['app_name'].notnull())]

    # Ordenar el DataFrame por 'sentiment' de mayor a menor
    top_3_df = top_3_df.sort_values(by='sentiment', ascending=False).head(3)

    # Crear una lista de diccionarios con el top 3
    top_3_dict = {}
    for i, row in top_3_df.iterrows():
        puesto = "Puesto " + str(i + 1 - top_3_df.index[0])
        top_3_dict[puesto] = row['app_name']

    return top_3_dict



In [99]:
# Ejemplo de uso:
posted = 2010
top_3_juegos = UsersNotRecommend(posted)
top_3_juegos

{'Puesto 1': 'Team Fortress 2'}